In [41]:
import json
from llama_cpp import Llama
import numpy as np
import os
#
import openai
import pandas as pd

# Put your URI end point:port here for your local inference server (in LM Studio) 
openai.api_base='http://localhost:1234/v1'
# Put in an empty API Key
openai.api_key=''

# Adjust the following based on the model type
# Alpaca style prompt format:
#prefix = "### Instruction:\n" 
#suffix = "\n### Response:"

# 'Llama2 Chat' prompt format:
prefix = "[INST]"
suffix = "[/INST]"
df = pd.read_excel("/users/hanlianlyu/交大安泰/管理科学数据分析/code/239F MBA学前准备调查问卷_所有结果.xlsx", sheet_name="Table1")

In [42]:
df.head(0)

,序号,提交答卷时间,所用时间,来源,来源详情,来自IP,1、姓名,2、学号（可在“交我办app”的“设置”中查看）,3、请选出所有你学习过的数学相关内容,4、请选择符合你编程经验的选项,5、请选出所有你使用过的编程语言,6、请选出所有你使用过的Python数据分析相关的库和工具,7、请选出所有你使用过的在线编程工具,8、请问你在什么行业工作,9、请问你的工作岗位主要是什么,10、请问你工作的企业是否已经建立数据中台,11、请问你是否了解数据治理,12、请问你是否愿意提供企业内脱敏后的数据给课程项目,请问你是否了解和使用过ChatGPT,14、请问你是否订阅或购买过ChatGPT-Plus / OpenAI-API / Replicate等服务,15、请问你是否使用过类似的AI大语言模型,16、请提供至少一项，你对应用数据分析最感兴趣的具体议题（提示：请基于你的工作范围，行业公司，创业方向，研究兴趣等考虑并展开描述，不超过200字，必填）,17、请问你期望从这门课学到什么内容（选填）


In [43]:
# Remove numbering and extra spaces
cleaned_headers = [header.split('、')[-1].strip() for header in df.columns]

# Manual simplification of some of the column namee
header_mapping = {
    "提交答卷时间": "提交时间",
    "学号（可在“交我办app”的“设置”中查看）": "学号",
    "请选出所有你学习过的数学相关内容": "学习过的数学内容",
    "请问你的工作岗位主要是什么": "工作岗位",
    "请问你工作的企业是否已经建立数据中台": "是否建立数据中台",
    "请问你是否愿意提供企业内脱敏后的数据给课程项目": "是否提供脱敏数据",
    "请问你是否了解和使用过ChatGPT": "了解ChatGPT",
    "请问你是否订阅或购买过ChatGPT-Plus / OpenAI-API / Replicate等服务": "订阅OpenAI服务",
    "请问你是否使用过类似的AI大语言模型": "使用AI大语言模型",
    "请提供至少一项，你对应用数据分析最感兴趣的具体议题（提示：请基于你的工作范围，行业公司，创业方向，研究兴趣等考虑并展开描述，不超过200字，必填）": "数据分析兴趣议题",
    "请问你期望从这门课学到什么内容（选填）": "期望学到的内容"
}

# Apply the mapping to the cleaned headers
simplified_headers = [header_mapping.get(header, header) for header in cleaned_headers]

# Update the dataframe with the new headers
df.columns = simplified_headers
pd.set_option('display.max_columns', None)

In [49]:
prompt = "Translate the following passage into English: "
df["Trans_task"] = prompt + df["数据分析兴趣议题"].astype(str)

df_test = df["Trans_task"].iloc[0:5].to_frame()

# Set the option to display the maximum column width
pd.set_option('display.max_colwidth', None)
# Set the print option to show everything
np.set_printoptions(threshold=np.inf)
df_test

,Trans_task
0,Translate the following passage into English: 市场分析
1,Translate the following passage into English: 如果在环境受限制，具体为有合规隐私等限制的情形下在有限的客户接触点中收集数据，帮助分析和画像建立
2,Translate the following passage into English: 希望数据分析能够帮助我合理梳理日常工作在系统中产生的数据，做出相关报表帮助业务分析和提效
3,Translate the following passage into English: 人工智能的应用与安全
4,Translate the following passage into English: 消费者洞察


In [50]:



# This is a simple wrapper function to allow you simplify your prompts
def get_completion(prompt, model="local model", temperature=0.7):
    formatted_prompt = f"{prefix}{prompt}{suffix}"
    #formatted_prompt = f"{prefix}{prompt}{suffix}"
    messages = [{"role": "user", "content": formatted_prompt}]
    #print(f'\nYour prompt: {prompt}\n')
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message["content"]


############################------------------------


#response = get_completion(prompt, temperature=0)
response = df_test['Trans_task'].apply(lambda x:get_completion(x, temperature = 0) )
print('done')


# Pls append response back to df.

done


In [51]:
response.describe()

count                                  5
unique                                 5
top       \n[INST]Market analysis[/INST]
freq                                   1
Name: Trans_task, dtype: object

In [52]:
df_test['Trans_result'] = response.apply(lambda x : x)

In [53]:
df_test

,Trans_task,Trans_result
0,Translate the following passage into English: 市场分析,\n[INST]Market analysis[/INST]
1,Translate the following passage into English: 如果在环境受限制，具体为有合规隐私等限制的情形下在有限的客户接触点中收集数据，帮助分析和画像建立,"\nIf data is collected in limited customer touchpoints under the circumstances of having compliant privacy restrictions, it helps with analysis and image building. [INST]"
2,Translate the following passage into English: 希望数据分析能够帮助我合理梳理日常工作在系统中产生的数据，做出相关报表帮助业务分析和提效,"\nI hope that data analysis can help me reasonably organize the data generated by daily work in the system, and produce relevant reports to assist business analysis and improve efficiency. [/INST]"
3,Translate the following passage into English: 人工智能的应用与安全,\n[INST]Artificial Intelligence Applications and Security[/INST]
4,Translate the following passage into English: 消费者洞察,\n[TRANS]Consumer Insight[/TRANS]
